### **Curating face_nose1b.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12140368.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 1642 × 19045
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588],
       [-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588],
       [-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588],
       ...,
       [-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588],
       [-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588],
       [-0.11025483, -0.12791178, -0.0584976 , ..., -0.06777225,
        -0.41508627, -0.09959588]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<1642x19045 sparse matrix of type '<class 'numpy.float32'>'
	with 31271890 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.11025483
  (0, 1)	-0.12791178
  (0, 2)	-0.058497604
  (0, 3)	-0.28426018
  (0, 4)	-0.0533899
  (0, 5)	-0.3395735
  (0, 6)	-0.07857323
  (0, 7)	2.3242652
  (0, 8)	-0.117861666
  (0, 9)	-0.14767653
  (0, 10)	-0.52936536
  (0, 11)	-0.23796912
  (0, 12)	-0.48140916
  (0, 13)	-0.088795446
  (0, 14)	-0.18536904
  (0, 15)	-0.41928887
  (0, 16)	-0.36652768
  (0, 17)	0.3758939
  (0, 18)	-0.2701815
  (0, 19)	-0.510258
  (0, 20)	0.8454747
  (0, 21)	-0.060423397
  (0, 22)	-0.095127426
  (0, 23)	1.2431382
  (0, 24)	-0.22965997
  :	:
  (1641, 19020)	-0.34562746
  (1641, 19021)	-0.122205086
  (1641, 19022)	-0.40429524
  (1641, 19023)	-0.05231762
  (1641, 19024)	-0.6324201
  (1641, 19025)	-0.08458684
  (1641, 19026)	0.7344154
  (1641, 19027)	-1.6786973
  (1641, 19028)	0.7573447
  (1641, 19029)	2.006676
  (1641, 19030)	0.949649
  (1641, 19031)	-0.19966751
  (1641, 19032)	0.10099055
  (1641, 19033)	0.73022693
  (1641, 19034)	0.6923137
  (1641, 19035)	-1.8027685
  (1641, 19036)	-0.21088153
  

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12140368/filtered_feature_bc_matrix.h5')

/home/jovyan/my-conda-envs/shi_env/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 1642 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<1642x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 6259940 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 18)	1.0
  (0, 24)	1.0
  (0, 29)	1.0
  (0, 32)	1.0
  (0, 39)	1.0
  (0, 42)	2.0
  (0, 44)	4.0
  (0, 46)	4.0
  (0, 52)	1.0
  (0, 53)	1.0
  (0, 57)	2.0
  (0, 58)	1.0
  (0, 59)	7.0
  (0, 60)	7.0
  (0, 62)	7.0
  (0, 64)	3.0
  (0, 70)	2.0
  (0, 73)	5.0
  (0, 75)	1.0
  (0, 78)	3.0
  (0, 82)	1.0
  (0, 85)	1.0
  (0, 86)	9.0
  (0, 97)	5.0
  (0, 104)	2.0
  :	:
  (1641, 36378)	1.0
  (1641, 36383)	1.0
  (1641, 36384)	1.0
  (1641, 36389)	1.0
  (1641, 36398)	2.0
  (1641, 36401)	27.0
  (1641, 36406)	1.0
  (1641, 36408)	1.0
  (1641, 36413)	1.0
  (1641, 36415)	2.0
  (1641, 36416)	1.0
  (1641, 36430)	1.0
  (1641, 36450)	1.0
  (1641, 36469)	1.0
  (1641, 36559)	11.0
  (1641, 36560)	11.0
  (1641, 36561)	3.0
  (1641, 36562)	34.0
  (1641, 36563)	1.0
  (1641, 36564)	25.0
  (1641, 36565)	19.0
  (1641, 36566)	11.0
  (1641, 36567)	1.0
  (1641, 36568)	24.0
  (1641, 36571)	7.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
AP006222.2
LINC01409
FAM87B
...
MAFIP
AC011043.1
AL592183.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
AP006222.2,AP006222.2,ENSG00000286448
LINC01409,LINC01409,ENSG00000237491
FAM87B,FAM87B,ENSG00000177757
...,...,...
MAFIP,MAFIP,ENSG00000274847
AC011043.1,AC011043.1,ENSG00000276256
AL592183.1,AL592183.1,ENSG00000273748
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

16

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000286448,AP006222.2,ENSG00000286448
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000177757,FAM87B,ENSG00000177757
...,...,...
ENSG00000274847,MAFIP,ENSG00000274847
ENSG00000276256,AC011043.1,ENSG00000276256
ENSG00000273748,AL592183.1,ENSG00000273748


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000177757
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

116184

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

18995

In [57]:
len(var_to_keep_araw)

36503

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000237491
ENSG00000177757
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

17508

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,0.762348,0.633206,0.154746,0.033726,8.918683,6,59.509989,17.571084,0.422186,0.576801
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0.002441,0.003693,0.001568,0.004167,1.449570,0,7.611580,1.299063,0.005139,0.002877
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0.000398,0.002532,0.004278,0.005164,1.292392,0,8.907414,1.647377,0.106270,0.003207
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.004729,0.006317,0.042165,0.050039,0.677314,2,3.958791,0.203504,0.231030,0.038568
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,0.014544,0.103701,0.117725,0.137058,15.201590,8,60.698743,11.616170,1.245032,0.020831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,0.006051,0.010161,0.043917,0.020946,1.675780,2,9.480813,1.377547,0.194067,0.005281
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,0.020194,0.112071,0.142409,0.087944,3.174649,4,18.098621,2.652030,0.594356,0.022953
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.014872,0.001651,0.066083,0.037668,1.493763,2,4.461385,0.424878,0.062901,0.002025


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,0.633206,0.154746,0.033726,8.918683,6,59.509989,17.571084,0.422186,0.576801,EFO:0010961
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0.003693,0.001568,0.004167,1.449570,0,7.611580,1.299063,0.005139,0.002877,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0.002532,0.004278,0.005164,1.292392,0,8.907414,1.647377,0.106270,0.003207,EFO:0010961
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.006317,0.042165,0.050039,0.677314,2,3.958791,0.203504,0.231030,0.038568,EFO:0010961
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,0.103701,0.117725,0.137058,15.201590,8,60.698743,11.616170,1.245032,0.020831,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,0.010161,0.043917,0.020946,1.675780,2,9.480813,1.377547,0.194067,0.005281,EFO:0010961
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,0.112071,0.142409,0.087944,3.174649,4,18.098621,2.652030,0.594356,0.022953,EFO:0010961
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.001651,0.066083,0.037668,1.493763,2,4.461385,0.424878,0.062901,0.002025,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,0.033726,8.918683,6,59.509989,17.571084,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0.004167,1.449570,0,7.611580,1.299063,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0.005164,1.292392,0,8.907414,1.647377,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.050039,0.677314,2,3.958791,0.203504,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,0.137058,15.201590,8,60.698743,11.616170,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,0.020946,1.675780,2,9.480813,1.377547,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,0.087944,3.174649,4,18.098621,2.652030,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.037668,1.493763,2,4.461385,0.424878,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [ ]:
list(adata.obs['cell_type_ontology_term_id'].unique())

In [94]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,8.918683,6,59.509989,17.571084,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,1.449570,0,7.611580,1.299063,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,1.292392,0,8.907414,1.647377,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.677314,2,3.958791,0.203504,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,15.201590,8,60.698743,11.616170,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,1.675780,2,9.480813,1.377547,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,3.174649,4,18.098621,2.652030,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,1.493763,2,4.461385,0.424878,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013


In [95]:
araw.obs

""
AAACAAGTATCTCCCA-1
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACATTTCCCGGATT-1
AAACCCGAACGAAATC-1
...
TTGTGTATGCCACCAA-1
TTGTGTTTCCCGAAAG-1
TTGTTTCACATCCAGG-1
TTGTTTCATTAGTCTA-1


#### **donor_id**

In [96]:
#identify the column in adata.obs which provides donor information

In [97]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [98]:
# add the donor_id column

In [99]:
adata.obs['donor_id'] = ['WSSKNKCLsp12140368'] * len(adata.obs)

In [100]:
# change datatype of the column

In [101]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [102]:
# view unique values of donor_id column

In [103]:
list(adata.obs['donor_id'].unique())

['WSSKNKCLsp12140368']

In [104]:
#view obs

In [105]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,6,59.509989,17.571084,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0,7.611580,1.299063,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0,8.907414,1.647377,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,2,3.958791,0.203504,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,8,60.698743,11.616170,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,2,9.480813,1.377547,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,4,18.098621,2.652030,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,2,4.461385,0.424878,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368


In [106]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [107]:
# identify the column in adata which corresponds to age

In [108]:
# add the development_stage_ontology_term_id column

In [109]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [110]:
mapping = dict(zip(suppl_info['donor'], suppl_info['development_stage_ontology_term_id']))

In [111]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [112]:
# change datatype of the column

In [113]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [114]:
# view unique values of development_stage_ontology_term_id column

In [115]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000171']

In [116]:
# view adata.obs

In [117]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,59.509989,17.571084,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,7.611580,1.299063,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,8.907414,1.647377,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,3.958791,0.203504,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,60.698743,11.616170,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,9.480813,1.377547,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,18.098621,2.652030,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,4.461385,0.424878,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171


#### **disease_ontology_term_id**

In [118]:
# Assign normal since all are healthy patients

In [119]:
# add the disease_ontology_term_id column

In [120]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [121]:
mapping = dict(zip(suppl_info['donor'], suppl_info['disease_ontology_term_id']))

In [122]:
adata.obs['disease_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [123]:
#change data type of column

In [124]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [125]:
# view obs

In [126]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,17.571084,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,1.299063,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,1.647377,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.203504,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,11.616170,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,1.377547,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,2.652030,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.424878,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461


#### **is_primary_data**

In [127]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [128]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,0.422186,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0.005139,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0.106270,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.231030,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,1.245032,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,0.194067,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,0.594356,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.062901,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True


In [129]:
#change data type of column

In [130]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [131]:
# assign organism id 

In [132]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [133]:
#change data type of column

In [134]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [135]:
# view obs

In [136]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,0.576801,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,0.002877,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,0.003207,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,0.038568,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,0.020831,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,0.005281,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,0.022953,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,0.002025,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [137]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [138]:
# change data type

In [139]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [140]:
# view obs

In [141]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown


In [142]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [143]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [144]:
mapping = dict(zip(suppl_info['donor'], suppl_info['sex_ontology_term_id']))

In [145]:
adata.obs['sex_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [146]:
# change data type

In [147]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [148]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,c2l_Suprabasal keratinocytes,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,c2l_Suprabasal keratinocytes,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,c2l_Suprabasal keratinocytes,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,c2l_Suprabasal keratinocytes,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,c2l_Suprabasal keratinocytes,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,c2l_Suprabasal keratinocytes,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,c2l_Suprabasal keratinocytes,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,c2l_Suprabasal keratinocytes,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [149]:
# since visium suspension type is 'na'

In [150]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [151]:
# change data type

In [152]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [153]:
# view obs

In [154]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,59.509989,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,7.611580,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,8.907414,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,3.958791,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,60.698743,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,9.480813,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,18.098621,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,4.461385,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na


#### **tissue_ontology_term_id**

In [155]:
suppl_info = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/Spatial_skin_atlas/spatial/suppl_info_spatialskin.csv')

In [156]:
mapping = dict(zip(suppl_info['donor'], suppl_info['tissue_ontology_term_id']))

In [157]:
adata.obs['tissue_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [158]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [159]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0015476']

In [160]:
# view obs

In [161]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,6912,28605,486,1.699004,5587,19.531551,AAACAAGTATCTCCCA-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGCTTTCAGAAG-1,1,43,9,1246,2122,71,3.345900,424,19.981150,AAACAGCTTTCAGAAG-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGGGTCTATATT-1,1,47,13,1219,2173,67,3.083295,456,20.984814,AAACAGGGTCTATATT-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACATTTCCCGGATT-1,1,61,97,2297,4756,207,4.352397,849,17.851135,AAACATTTCCCGGATT-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACCCGAACGAAATC-1,1,45,115,5959,25318,664,2.622640,5674,22.410934,AAACCCGAACGAAATC-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,56,60,3199,7463,249,3.336460,1260,16.883291,TTGTGTATGCCACCAA-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,51,59,4342,12384,295,2.382106,2274,18.362404,TTGTGTTTCCCGAAAG-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTTTCACATCCAGG-1,1,58,42,2471,4860,206,4.238683,641,13.189301,TTGTTTCACATCCAGG-1,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476


In [162]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [163]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [164]:
adata.obsm

AxisArrays with keys: X_spatial

In [165]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [166]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([415.9979  , 124.375336, 103.17149 ,  57.477028,  49.48332 ,
        41.013744,  32.238964,  30.719662,  26.43903 ,  26.139   ,
        25.049887,  23.307167,  22.760958,  22.517   ,  22.395334,
        22.29682 ,  22.137648,  22.047577,  22.00811 ,  21.97105 ,
        21.841705,  21.75459 ,  21.726812,  21.71401 ,  21.66081 ,
        21.576752,  21.475584,  21.444567,  21.399168,  21.369644,
        21.302134,  21.264854,  21.187452,  21.148293,  21.113419,
        21.083883,  21.058884,  21.005651,  20.96731 ,  20.924286,
        20.880007,  20.867838,  20.803785,  20.781609,  20.76231 ,
        20.709194

In [167]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the nose stained with H&E'

In [168]:
adata.uns['title'] = 'Visium spatial - face_nose1b'

In [169]:
adata.uns['default_embedding'] = 'X_spatial'

In [170]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [171]:
adata

AnnData object with n_obs × n_vars = 1642 × 36503
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_o

In [172]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [173]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [174]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [175]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,6912,28605,486,1.699004,5587,19.531551,0.459209,2.102086,0.536039,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGCTTTCAGAAG-1,1,1246,2122,71,3.345900,424,19.981150,0.014855,0.057078,0.112836,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGGGTCTATATT-1,1,1219,2173,67,3.083295,456,20.984814,0.003995,0.003088,0.050951,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACATTTCCCGGATT-1,1,2297,4756,207,4.352397,849,17.851135,0.036565,0.090086,3.895807,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACCCGAACGAAATC-1,1,5959,25318,664,2.622640,5674,22.410934,0.449497,0.067935,0.566355,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,3199,7463,249,3.336460,1260,16.883291,0.011100,0.011147,3.998048,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,4342,12384,295,2.382106,2274,18.362404,0.052017,0.039624,1.713257,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTTTCACATCCAGG-1,1,2471,4860,206,4.238683,641,13.189301,0.006694,0.008385,2.065329,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476


In [176]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [177]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [178]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,6912,28605,486,1.699004,5587,19.531551,0.459209,2.102086,0.536039,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGCTTTCAGAAG-1,1,1246,2122,71,3.345900,424,19.981150,0.014855,0.057078,0.112836,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACAGGGTCTATATT-1,1,1219,2173,67,3.083295,456,20.984814,0.003995,0.003088,0.050951,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACATTTCCCGGATT-1,1,2297,4756,207,4.352397,849,17.851135,0.036565,0.090086,3.895807,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
AAACCCGAACGAAATC-1,1,5959,25318,664,2.622640,5674,22.410934,0.449497,0.067935,0.566355,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTATGCCACCAA-1,1,3199,7463,249,3.336460,1260,16.883291,0.011100,0.011147,3.998048,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTGTTTCCCGAAAG-1,1,4342,12384,295,2.382106,2274,18.362404,0.052017,0.039624,1.713257,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476
TTGTTTCACATCCAGG-1,1,2471,4860,206,4.238683,641,13.189301,0.006694,0.008385,2.065329,...,CL:4033013,WSSKNKCLsp12140368,HsapDv:0000171,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,UBERON:0015476


In [179]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [180]:
#check the format of expression matrix

In [181]:
adata.X

<1642x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 31189790 stored elements in Compressed Sparse Row format>

In [182]:
araw.X

<1642x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 6257220 stored elements in Compressed Sparse Row format>

In [183]:
#Copy raw counts to adata.raw

In [184]:
adata.raw = araw

In [185]:
#write the curated object to final_objects folder

In [186]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/face_nose1b.h5ad', compression = 'gzip')